# ACS Age Distribution in Phoenix Urban Villages, City of Phoenix, U.S., Maricopa County, Arizona, and Comparable Cities

For population by age 1-year data
- https://www.census.gov/data/developers/data-sets/acs-1year.html
- https://api.census.gov/data/2010/acs/acs1/variables.html
- https://api.census.gov/data/2021/acs/acs1/variables.html

For population by age 5-year data
- https://www.census.gov/data/developers/data-sets/acs-5year.html
- https://api.census.gov/data/2013/acs/acs5/variables.html
- https://api.census.gov/data/2021/acs/acs5/variables.html

** Note: Census Block Groups (which are aggregated to Phoenix Urban Village areas) are not available until 2013. 

In [35]:
import pandas as pd
import math
import numpy as np

In [36]:
import getters as get
import utilcalcs as calc
import geo_agg
from acs_age_vars import *

In [37]:
#Search parameters
y1 = '2021'
y0_5 = '2013' #for urban villages
y0_1 = '2010'

cols = 'group(B01001)'

source5 = 'acs/acs5'
source1 = 'acs/acs1'

var_cols = ['GEO_ID','B01001_001E','B01001_001M']+age_vars

## Population by Age - ACS 1 Year for larger geos
##### <10 years, 10-24, 25-39, 40-54, 55+
Note - does not include Indianapolis, which is not available as a consolidated city in the 1-year data set

In [38]:
def group_age(df,year):
    df.rename(columns={'B01001_001E':'tot_E','B01001_001M':'tot_M'},inplace=True)
    df['p_u10_E'] = df.loc[:,u10_e].sum(axis=1)
    df['p_u10_M'] = df.apply(lambda x: (calc.get_moe(x[u10_m])),axis=1)
    df['p_1024_E'] = df.loc[:,genz_e].sum(axis=1)
    df['p_1024_M'] = df.apply(lambda x: (calc.get_moe(x[genz_m])),axis=1)
    df['p_2539_E'] = df.loc[:,mil_e].sum(axis=1)
    df['p_2539_M'] = df.apply(lambda x: (calc.get_moe(x[mil_m])),axis=1)
    df['p_4054_E'] = df.loc[:,genx_e].sum(axis=1)
    df['p_4054_M'] = df.apply(lambda x: (calc.get_moe(x[genx_m])),axis=1)
    df['p_o55_E'] = df.loc[:,boom_e].sum(axis=1)
    df['p_o55_M'] = df.apply(lambda x: (calc.get_moe(x[boom_m])),axis=1)
    df = df.drop(columns=age_vars)
    
    #add end-year into column name
    for col in df.columns[1:]:
        df.rename(columns={col:f'{col[:-1]}{year[-2:]}{col[-1:]}'},inplace=True)
    return df

## City of Phoenix, Maricopa, AZ, U.S., other cities

In [39]:
def year_table(source,year,cols):
    ph = get.get_phx(source,year,cols)
    mar = get.get_maricopa(source,year,cols)
    us = get.get_us(source,year,cols)
    az = get.get_az(source,year,cols)
    cit = get.get_comp_cities(source,year,cols)
    Y = pd.concat([ph,mar,us,az,cit])
    Y = Y[var_cols]
    Y = get.clean_data(Y,Y.columns)
    Y = group_age(Y,year)
    Y = geo_agg.sumgeo_cv(Y,'GEO_ID')
    return Y

In [40]:
Y1 = year_table(source1,y1,cols)
Y0 = year_table(source1,y0_1,cols)

In [48]:
Y0Y1 = pd.merge(Y0,Y1,how='left',on='GEO_ID')

In [49]:
def calc_change(df,yr0,yr1,var_list):
    for var in var_list:
        df[f'{var}_{yr0[-2:]}{yr1[-2:]}E'] = df[f'{var}_{yr1[-2:]}E'] - df[f'{var}_{yr0[-2:]}E']
        df[f'{var}_{yr0[-2:]}{yr1[-2:]}M'] = df.apply(lambda x: (calc.get_moe([x[f'{var}_{yr0[-2:]}M'],\
                                                                                x[f'{var}_{yr1[-2:]}M']])),axis=1)
        df[f'{var}_{yr0[-2:]}{yr1[-2:]}C'] = df.apply(lambda x: (calc.get_cv(x[f'{var}_{yr0[-2:]}{yr1[-2:]}E'],\
                                                                            x[f'{var}_{yr0[-2:]}{yr1[-2:]}M'])),axis=1)
        for  y  in [yr0[-2:],yr1[-2:]]:
            df[f'{var}_{y}P'] = df[f'{var}_{y}E'] / df[f'tot_{y}E']
    return df

In [50]:
change_vars = ['tot','p_u10','p_1024','p_2539','p_4054','p_o55']

Y0Y1 = calc_change(Y0Y1,y0_1,y1,change_vars)

In [51]:
Y0Y1

,GEO_ID,p_u10_10E,p_u10_10M,p_u10_10C,p_4054_10E,p_4054_10M,p_4054_10C,p_o55_10E,p_o55_10M,p_o55_10C,...,p_4054_1021E,p_4054_1021M,p_4054_1021C,p_4054_10P,p_4054_21P,p_o55_1021E,p_o55_1021M,p_o55_1021C,p_o55_10P,p_o55_21P
0,0455000,220765.0,6608.957028,1.819855,292753.0,7921.511661,1.644905,267029.0,7834.840968,1.783634,...,22098.0,11283.238232,31.039512,0.201971,0.193809,102681.0,12582.401917,7.449164,0.184224,0.227578
1,04013,562409.0,7872.068216,0.850885,762589.0,7050.707979,0.562052,866348.0,11920.184059,0.836421,...,84650.0,9711.135207,6.973923,0.199246,0.188418,371163.0,18927.741519,3.100047,0.226356,0.275211
2,0100000US,40525402.0,60387.037061,0.090584,65914697.0,61977.622534,0.057159,77195489.0,105966.835085,0.083447,...,-3878817.0,98024.267602,1.536273,0.213075,0.186915,21511559.0,168350.595449,0.475748,0.249541,0.297406
3,0400000US04,904399.0,9523.537631,0.640136,1249064.0,9375.584302,0.456297,1620055.0,17087.890156,0.641199,...,46856.0,14248.784088,18.486159,0.194748,0.178101,593941.0,26348.646550,2.696802,0.252591,0.304274
4,0473000,19534.0,2790.119531,8.682918,21984.0,2439.773760,6.746475,29798.0,2534.892503,5.171381,...,4564.0,3649.868628,48.614486,0.135601,0.144197,2916.0,3972.393611,82.813064,0.183799,0.177688
5,0465000,18354.0,2557.891905,8.471992,53451.0,3767.460949,4.284765,77461.0,4682.154632,3.674487,...,-9552.0,5244.678922,33.377875,0.245214,0.180837,26748.0,6881.045342,15.638576,0.355363,0.429278
6,0427820,35555.0,3174.677464,5.427915,44534.0,3669.596981,5.009112,44498.0,3439.067897,4.698231,...,2503.0,5120.276457,124.355975,0.195876,0.188429,19558.0,5361.878495,16.665818,0.195718,0.256607
7,4865000,198777.0,5686.740191,1.739127,255360.0,4913.128433,1.169605,277706.0,6739.814760,1.475356,...,3556.0,8586.343343,146.784635,0.191373,0.178334,63245.0,11287.125719,10.849039,0.208119,0.234837
8,4819000,194791.0,6500.839715,2.028779,231995.0,6538.375180,1.713268,216916.0,6721.048505,1.883560,...,2260.0,10555.353523,283.921605,0.192880,0.181813,73751.0,11089.819881,9.140936,0.180343,0.225596
9,1235000,107521.0,3174.403881,1.794746,174991.0,3681.648408,1.278971,183392.0,5436.047093,1.801926,...,3517.0,5925.573896,102.421740,0.212544,0.186993,76282.0,9515.290852,7.582878,0.222748,0.272017


#### Urban Villages